In [0]:
!unzip -d "/content/gdrive/My Drive/UAV/data1" "/content/gdrive/My Drive/UAV/data.zip"

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39326_aug_0_7327_34588.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39326_aug_0_7353_34589.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39326_aug_0_8099_34590.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39326_aug_0_8306_34591.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39326_aug_0_8335_34592.png  
 extracting: /content/gdrive/My Drive/UAV/data1/triangle_5_39326_aug_0_9856_34593.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39326_aug_0_9996_34594.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39337_34595.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39337_aug_0_2248_34596.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39337_aug_0_2811_34597.png  
  inflating: /content/gdrive/My Drive/UAV/data1/triangle_5_39337_aug_0_3016_3459

In [0]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%tensorflow_version 1.x
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import multiprocessing
np.random.seed(1000)

In [0]:
data_path = "/content/gdrive/My Drive/UAV/data1"
models_path = "/content/gdrive/My Drive/UAV/models/"

In [0]:
shape_dict = {'circle': 0, 'semicircle': 1, 'quartercircle': 2, 'triangle': 3, 'square': 4, 'rectangle': 5,
              'trapezoid': 6, 'pentagon': 7, 'hexagon': 8, 'heptagon': 9, 'octagon': 10, 'star': 11, 'cross': 12}

In [0]:
name = "alex"
batch_size = 128
num_classes = 13
epochs = 10

# input image dimensions
img_rows, img_cols = 244, 244

checkpoint_path = "models/cp_" + name + ".ckpt"  # https://www.tensorflow.org/tutorials/keras/save_and_load

In [0]:
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(img_rows,img_cols,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(num_classes))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 59, 59, 96)        34944     
_________________________________________________________________
activation_28 (Activation)   (None, 59, 59, 96)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 29, 29, 96)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 19, 19, 256)       2973952   
_________________________________________________________________
activation_29 (Activation)   (None, 19, 19, 256)       0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 9, 9, 256)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 7, 7, 384)        

In [0]:
print('START PREPROCESSING')

cpu_num = multiprocessing.cpu_count()
input_images = os.listdir(data_path)
x_in = np.array_split(input_images, cpu_num)
names = ["proc_" + str(n) for n in range(cpu_num)]

def preproccessing(iterable):
  images, name = iterable
  x = np.zeros((len(images), 244, 244, 3), 'float32')
  y = np.zeros(len(images))

  for i, image_entry in enumerate(images):
    print(name + " ->" + image_entry + " --- " + str(100 * i / len(images)) + "%")

    image = cv2.imread(image_entry)

    x[i] = image
    shape_name, _ = os.path.splitext(image_entry)
    shape_name = shape_name.split("_")[0]
    y[i] = shape_dict[shape_name]

    print(f'X: {x.shape}')
    print(f'Y shape: {y.shape}')
    print(f'Y: {np.unique(y)}')
    print('END PREPROCESSING')

    # serialization
    with open("/content/gdrive/My Drive/UAV/arrays/" + name + ".npy", 'wb') as f:
      np.save(f, x)
      np.save(f, y)

if __name__ ==  '__main__':
  with Pool(processes=cpu_num) as p:
    p.map(preprocessing, zip(x_in, names))


In [0]:
# deserialization
xx = []
yy = []

for archive in os.listdir("/content/gdrive/My Drive/UAV/arrays/"):
  with open("/content/gdrive/My Drive/UAV/arrays/" + archive + ".npy", 'wb') as f:
    xx += [np.load(f)]
    yy += [np.load(f)]

X = np.concatenate(xx)
Y = np.concatenate(yy)

In [0]:
x_train, x_test_val, y_train, y_test_val = train_test_split(X, Y, test_size=0.4, random_state=42, shuffle=True)
x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size=0.5, random_state=42,
                                                    shuffle=True)
del x_test_val, y_test_val

NameError: ignored

In [0]:
x_train = x_train/255
x_test = x_test/255
x_val = x_val/255

In [0]:
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_val, y_val),
              callbacks=[cp_callback])

In [0]:
model.save_weights(models_path + name)
model.save(models_path + name + ".h5")

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])